In [1]:
import numpy as np
import scipy as sp
import pandas as pd

import re

from IPython.display import display

# Aula 03 &mdash; Introdução a Pandas - Parte 2

Renato Vimieiro
rv2 {em} cin.ufpe.br

março 2017

# Resumo

- Nessa aula continuaremos a explorar Pandas
- Continuaremos a usar os dados do MovieLens como exemplo
- Reutilizaremos os dados descritos na aula 02

In [48]:
usuarios = pd.read_csv(
    "http://files.grouplens.org/datasets/movielens/ml-100k/u.user",
    sep='|',header=None, names=["user_id", "age", "gender", "occupation", "zip_code"])
filmes = pd.read_csv(
    "http://files.grouplens.org/datasets/movielens/ml-100k/u.item",
    sep='|',header=None, names=["movie_id", "movie_title",  "release_date", "video_release_date", "IMDb_URL", "unknown", "Action", "Adventure", "Animation", 
        "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy","FilmNoir", "Horror", "Musical", "Mystery", "Romance", "SciFi","Thriller", "War", "Western"])
avaliacoes = pd.read_csv(
    "http://files.grouplens.org/datasets/movielens/ml-100k/u.data",
    sep='\t',header=None, names=["user_id", "movie_id", "rating", "timestamp"])


# Indexação e seleção de dados

In [3]:
# Selecionando uma coluna
print(usuarios['age'].head())
print(usuarios.age.head())

0    24
1    53
2    23
3    24
4    33
Name: age, dtype: int64
0    24
1    53
2    23
3    24
4    33
Name: age, dtype: int64


In [4]:
# Selecionando multiplas colunas

usuarios[['age','gender']].head()

,age,gender
0,24,M
1,53,F
2,23,M
3,24,M
4,33,F


In [5]:
# Selecionando linhas com slice
avaliacoes[10:15]

,user_id,movie_id,rating,timestamp
10,62,257,2,879372434
11,286,1014,5,879781125
12,200,222,5,876042340
13,210,40,3,891035994
14,224,29,3,888104457


In [6]:
# Nao e possivel combinar selecao de linha e coluna diretamente via []
avaliacoes[1:5,'rating']

TypeError: unhashable type: 'slice'

In [20]:
# A selecao de porcoes especificas sao feitas atraves dos atributos
# loc permite selecao por labels
print(avaliacoes.loc[10:15,'rating'])

# iloc permite a selecao por indice
display(filmes.iloc[:5,[1,2,4]])

10    2
11    5
12    5
13    3
14    3
15    3
Name: rating, dtype: int64


,movie_title,release_date,IMDb_URL
0,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995)


In [25]:
# Ainda podemos selecionar os dados com mascaras 
# booleanas como em NumPy
usuarios[(usuarios.age > 40) & 
         ~(usuarios.occupation.isin(['none','other']))].head()

,user_id,age,gender,occupation,zip_code
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
9,10,53,M,lawyer,90703
12,13,47,M,educator,29206
13,14,45,M,scientist,55106


Exercício:
======

- Mostre a média de idade das mulheres cientistas
- Quantos filmes de animação foram lançados em 1968

In [9]:
cientistas = usuarios[(usuarios.gender=='F') 
                      & (usuarios.occupation=='scientist')]
print(cientistas.head())
print(cientistas.age.mean())

     user_id  age gender occupation zip_code
174      175   26      F  scientist    21911
729      730   31      F  scientist    32114
929      930   28      F  scientist    07310
28.3333333333


In [27]:
print(filmes[(filmes.Animation==1)&
             filmes.release_date.str.contains('1968')].shape[0])

1


# Atribuições de valores

- A indexação através de `loc` e `iloc` retorna uma visão do data frame
- Essa visão pode ser modificada como fizemos em NumPy
- Além disso, podemos também acrescentar novos dados

In [11]:
c2 = cientistas.copy()
c2.iloc[0,1] = -1
c2.loc[:,'zip_code'] = None
c2.head()

,user_id,age,gender,occupation,zip_code
174,175,-1,F,scientist,None
729,730,31,F,scientist,None
929,930,28,F,scientist,None


In [49]:
filmes['release_year'] = filmes.release_date.apply(
        lambda x: not x is np.nan and 
    re.search("\d+\-\w+\-(\d+)",str(x)).group(1) or None)
filmes[['movie_title','release_date','release_year']].head()

,movie_title,release_date,release_year
0,Toy Story (1995),01-Jan-1995,1995
1,GoldenEye (1995),01-Jan-1995,1995
2,Four Rooms (1995),01-Jan-1995,1995
3,Get Shorty (1995),01-Jan-1995,1995
4,Copycat (1995),01-Jan-1995,1995


# Operações básicas

- Pandas conta com várias funções pré-definidas para obter estatísticas e informações básicas dos dados
- Ela conta também, como vimos anteriormente, com uma função `apply` que aplica uma função aos elementos
- Seguem alguns exemplos

In [13]:
print("Mediana de idade dos usuarios ", usuarios.age.median())

piorAvaliacao = avaliacoes.rating.argmin()
print(filmes[filmes.movie_id == avaliacoes.movie_id.iloc[piorAvaliacao]].movie_title.iloc[0])

avaliacoes.rating.value_counts()

Mediana de idade dos usuarios  31.0
Heavyweights (1994)


4    34174
3    27145
5    21201
2    11370
1     6110
Name: rating, dtype: int64

# Concat, merge e joins

- Vimos no exemplo anterior que pode ser necessário fazer cruzamento de dados em diferentes tabelas
- Essa é uma situação muito comum em bancos de dados, onde tabelas são frequentemente cruzadas
- Pandas dispõe de mecanismos similares para cruzamento de dados

In [4]:
# A forma mais simples de juntar diferentes dados e concatenacao
A = pd.Series(["A{}".format(a) for a in range(4)],index=range(4), name="A")
B = pd.Series(["B{}".format(a) for a in range(5)],index=range(5), name= "B")
C = pd.Series(["C{}".format(a) for a in range(5)],index=range(5), name= "C")
pd.concat([A,B,C],axis=1)

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2
3,A3,B3,C3
4,NaN,B4,C4


# Merge 

- Uma opção mais [SQL-like](https://www.codeproject.com/KB/database/Visual_SQL_Joins/Visual_SQL_JOINS_orig.jpg) é merge
- De acordo com a documentação essa função foi otimizada para desempenho
- A sintaxe é

```python
pd.merge(left, right, how='inner', 
on=None, left_on=None, right_on=None,
left_index=False, right_index=False, sort=True,
suffixes=('_x', '_y'),copy=True,indicator=False)
```         

In [54]:
avaliacaoFilmes = pd.merge(avaliacoes,
                           filmes[["movie_id","movie_title","release_year"]], 
                           on="movie_id")
avaliacaoFilmes.head()

,user_id,movie_id,rating,timestamp,movie_title,release_year
0,196,242,3,881250949,Kolya (1996),1997
1,63,242,3,875747190,Kolya (1996),1997
2,226,242,5,883888671,Kolya (1996),1997
3,154,242,3,879138235,Kolya (1996),1997
4,306,242,5,876503793,Kolya (1996),1997


Exercício:
======

- Calcule a média e desvio padrão das avaliações dadas aos filmes da década de 80 por programadores

In [56]:
avaliacaoFilmes.release_year.fillna(value=-1,inplace=True)
avaliacaoFilmes['release_year'] = avaliacaoFilmes.release_year.astype(np.int)
avalFilmUsu = pd.merge(avaliacaoFilmes,usuarios[["user_id","occupation"]], 
                       on="user_id")
selecao = avalFilmUsu[(1980 <= avalFilmUsu.release_year) & 
            (avalFilmUsu['release_year'] < 1990) 
            & (avalFilmUsu.occupation == 'programmer')]
selecao.rating.mean(), selecao.rating.std() 


(3.8215246636771298, 1.0184489421764065)

# Split-apply-combine (1/3)

- Um recurso muito útil de Pandas é o *group-by*
- Podemos agrupar dados a partir de um critério para aplicar determinada função
- Por exemplo, pode ser útil saber a média das avaliações por gênero do filme, ou faixa de idade


# Split-apply-combine (2/3)

- O procedimento é executado da seguinte forma:
    1. **Split**: dividir dados conforme um critério (e.g. coluna)
    2. **Apply**: aplicar uma determinada função a cada um dos grupos
    3. **Combine**: reunir resultados em uma única estrutura de dados

# Split-apply-combine (3/3)

- A função que será aplicada aos grupos pode ter como objetivo:
    - Sumarizar informações: computar estatísticas descritivas dos grupos (média, desvios, mínimo/máximo, contar valores, etc.)
    - Transformar valores: normalizar dados nos grupos; gerar novos atributos; preencher dados ausentes
    - Filtrar dados: eliminar grupos a partir de estatísticas computadas para o grupo

# Exemplos

In [11]:
usuarioPorProfissao = usuarios.groupby(usuarios.occupation,as_index=True)
usuarioPorProfissao.age.mean()
# usuarioPorProfissao.mean()

occupation
administrator    38.746835
artist           31.392857
doctor           43.571429
educator         42.010526
engineer         36.388060
entertainment    29.222222
executive        38.718750
healthcare       41.562500
homemaker        32.571429
lawyer           36.750000
librarian        40.000000
marketing        37.615385
none             26.555556
other            34.523810
programmer       33.121212
retired          63.071429
salesman         35.666667
scientist        35.548387
student          22.081633
technician       33.148148
writer           36.311111
Name: age, dtype: float64

In [78]:
print(filmes.release_year.dropna().astype(np.int).describe())
#pd.cut(filmes.release_year.dropna().astype(np.int),bins=10).head()
decada = pd.cut(filmes.release_year.fillna(value=0).astype(np.int),
       bins=[1900,1940,1950,1960,1970,1980,1990,2000],
       labels=['{}s'.format(a) for a in np.arange(30,100,10)])
filmePorDecada = filmes.groupby(decada)
filmePorDecada[filmes.columns[5:]].sum()


count    1681.00000
mean     1989.38608
std        14.25358
min      1922.00000
25%      1993.00000
50%      1995.00000
75%      1996.00000
max      1998.00000
Name: release_year, dtype: float64


,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
release_year,,,,,,,,,,,,,,,,,,,
30s,0,1,3,3,5,14,2,0,11,0,1,2,7,1,10,0,4,3,0
40s,0,0,3,3,5,4,4,0,16,0,10,1,5,9,6,0,7,3,2
50s,0,3,6,1,6,15,3,0,22,1,1,2,6,3,10,4,8,3,4
60s,0,3,3,3,7,9,1,0,15,0,2,3,4,3,4,5,5,5,4
70s,1,9,9,1,6,20,4,0,17,1,1,12,4,2,6,7,2,5,1
80s,0,33,18,5,5,33,6,4,53,2,2,15,5,2,16,15,17,10,3
90s,0,202,93,26,88,410,89,46,591,18,7,57,25,41,195,70,208,42,13


Exercício:
======
- Mostre a quantidade relativa de filmes de cada gênero por década
- Calcule a avaliação média de cada gênero por faixa etária

In [80]:
b = filmePorDecada[filmes.columns[5:]].sum()
sizes = filmePorDecada.size()
print(sizes)
b.T/sizes

release_year
30s      39
40s      44
50s      52
60s      41
70s      58
80s     123
90s    1324
dtype: int64


release_year,30s,40s,50s,60s,70s,80s,90s
unknown,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000
Action,0.025641,0.000000,0.057692,0.073171,0.155172,0.268293,0.152568
Adventure,0.076923,0.068182,0.115385,0.073171,0.155172,0.146341,0.070242
Animation,0.076923,0.068182,0.019231,0.073171,0.017241,0.040650,0.019637
Children,0.128205,0.113636,0.115385,0.170732,0.103448,0.040650,0.066465
Comedy,0.358974,0.090909,0.288462,0.219512,0.344828,0.268293,0.309668
Crime,0.051282,0.090909,0.057692,0.024390,0.068966,0.048780,0.067221
Documentary,0.000000,0.000000,0.000000,0.000000,0.000000,0.032520,0.034743
Drama,0.282051,0.363636,0.423077,0.365854,0.293103,0.430894,0.446375
Fantasy,0.000000,0.000000,0.019231,0.000000,0.017241,0.016260,0.013595
